In [2]:
import numpy as np
from DensityCanvas import DensityCanvas
from norms import Lp_norm, Lp_distance, KL_norm, JS_norm

# Intro
A density canvas is intended as a class to strandardize the way we work with densities.<br>
It is essentially a 2d canvas where we can define a scalar density function.<br>
It is supposed to automatize certain recurring operations: generating sum of gaussians, performing arithmetic operations between two densities, calculating norms, embedding registers, calculating QUBO coefficients, and more.<br>
This notebook presents the functionalities implemented so far.  


## Creating a canvas
To create a canvas, you need to supply:
1. An origin (coordinates of the bottom left corner)
2. The length in X and Y direction (as two separate variables)
3. The resolution in X and Y direction (as two separate variables)

In [ ]:
# this creates an empty canvas representing a 40x40 region, composed of 1200x1200=1440000 points
canvas = DensityCanvas(
    origin=(-20,-20),
    length_x=40,
    length_y=40,
    npoints_x=1200,
    npoints_y=1200,
    )

In [ ]:
# a canvas can be drawn, but at first it's just empty so it's just a flat 2d plot that is 0 everywhere
canvas.draw()

## Defining a density
You can define a density to put in a canvas. For now you can either pass an existing density (as a numpy array) that you got from somewhere else, or you can define it as a sum of Gaussians. To define a density as a sum of Gaussians you need to provide:
1. The centers of the Gaussians (as a list of coordinates)
2. The variance of the Gaussians (as a single number)
3. the amplitude of the gaussians (as a single number) 

In [ ]:
# define centers, variances, amplitudes
centers = np.array([[0,0], [5,5], [-3,-3]])
variance = 1
amplitude = 10

# assign the density to the canvas
canvas.set_density_from_gaussians(
    centers=centers,
    amplitude=amplitude,
    variance=variance
)

In [ ]:
# now you can plot it again to see the Gaussians

canvas.draw()

In [ ]:
# you can also choose to display the center of the Gaussians as red crosses
canvas.draw(draw_centers=True)

In [ ]:
# if you wish, you can delete every density related info from a canvas.
# plotting it will return an empty one.

canvas.clear_density()
canvas.draw()

## Arithmetic operations
You can perform binary arithmetic operations with canvases:
1. sum and difference (element-wise sum/difference between the densities)
2. product of two canvases (element-wise product between the two densities)
3. product between a number and a canvas (multiply density by a fixed value)
4. Taking powers of a canvas (element-wise power of density)

In order for the operations to be well defined, the two canvases need to be built with the same parameters (same origin, same length, same resolution)

In [ ]:
# define two test canvases

stg1 = DensityCanvas(
    origin=(-20,-20),
    length_x=40,
    length_y=40,
    npoints_x=1200,
    npoints_y=1200,
    )

stg2 = DensityCanvas(
    origin=(-20,-20),
    length_x=40,
    length_y=40,
    npoints_x=1200,
    npoints_y=1200,
    )


centers1 = np.array([[0,0], [5,5], [-3,-3]])
centers2 = np.array([[0,10], [-4,3], [-1,-5]])
variance = 10
amplitude = 10

stg1.set_density_from_gaussians(centers1, amplitude, variance)
stg2.set_density_from_gaussians(centers2, amplitude, variance)

stg1.draw()
stg2.draw()


In [ ]:
# sum
sum_stg = stg1 + stg2

# difference
diff_stg = stg1 - stg2

# canvas product
prod_stg = stg1*stg2

# scalar product
scal_prod_stg = 5*stg1

# power
pow_stg = stg1**4

In [ ]:
#visualize
sum_stg.draw()
diff_stg.draw()
prod_stg.draw()
scal_prod_stg.draw()
pow_stg.draw()

## Integral

It is possible to integrate a canvas (meaning taking the integral of the density in the whole region)

In [ ]:
# to integrate, either call integrate()
print(stg1.integrate())

# or cast a canvas to float
print(float(stg1))

## Norms

It is possible to calculate the norm of a canvas (meaning calculating a functional norm of the density). <br>

In [ ]:
# To calculate an Lp norm, choose the canvas and a p
print(
    Lp_norm(stg1, p=2)
)

# To measure an Lp distance between two canvases, either call norm on the difference
print(
    Lp_norm(stg1-stg2, p=2)
)

# Or call Lp_distance
print(
    Lp_distance(stg1, stg2, p=2)
)

## Lattices
The next step is to define a lattice on top of a density. <br>
It is possible to do so by either supplying a set of custom positions, or by creating a rectangular lattice with the helper function provided.<br>
The lattice can be drawn by specifying draw_lattice=True, and it is displayed as blue dots.

In [ ]:
# define a lattice with custom positions
pos = [[-5,-5], [2,3], [7,-1]]
stg1.define_custom_lattice(pos)
stg1.draw(draw_lattice=True)

In [ ]:
# clear existing lattice
stg1.clear_lattice()

# define a 5x4 rectangular lattice with lattice spacing 4
stg1.define_rectangular_lattice(xnum=5, ynum=4, spacing=4)
stg1.draw(draw_lattice=True)

In [ ]:
# lattice and centers can be displayed together
stg1.draw(draw_centers=True, draw_lattice=True)

# Cost functions, (Q)UBO coefficients
Once a density and a lattice has been defined, it is possible to calculate the coefficients of the (Q)UBO problem.<br>
The Q has been put in between parentheses because depending on the order of the Lp norm chosen, it can be more than quadratic.<br>

## Calculating (Q)UBO coefficients

In [ ]:
# define a canvas
canvas = DensityCanvas(
    origin=(-20,-20),
    length_x=40,
    length_y=40,
    npoints_x=100,
    npoints_y=100,
)

# define a base density as a sum of Gaussians
canvas.set_density_from_gaussians(
    centers = np.array([[0,0], [5,5], [-3,-3]]),
    amplitude = 10,
    variance = 10,
)

# define a small rectangular lattice (to make calculations faster)
canvas.define_rectangular_lattice(xnum=2, ynum=3, spacing=5)
canvas.draw(draw_centers=True, draw_lattice=True)

In [ ]:
# now decide the order of the Lp norm and the parameters of the test Gaussian mixture
mixture_parameters = [15, 15]   # test amplitude and variance
p = 4   # order of the Lp norm

# calculate the coefficients
canvas.calculate_ubo_coefficients(p, mixture_parameters)

In [ ]:
# the coefficients are stored, together with the mixture parameters and the Lp order, in a dictionary
for key, val in canvas._ubo.items():
    print(key, "=", val)

In [ ]:
# to see them better, print the coefficients out order by order
for i in range(1, p+1):
    print(f"Coefficients of order {i}:")
    print(canvas._ubo["coeffs"][i])
    print()

In [ ]:
# focus on the coefficients of order 2
# they are a dictionary where the key is of the type (i,j) and the value is a number
# this gives you the interaction strength between lattice point i and j
for pair, strength in canvas._ubo["coeffs"][2].items():
    print(f"interacting lattice points: {pair}      interaction strength: {strength}")

In [ ]:
# it is possible to truncate the calculation of the coefficients
# either from below (starting from terms of order > 1)
# or from above (ending with terms of order < p)
# by using the "high" and "low" arguments

# in this example, p=6 is used, but only coefficients of order 2,3,4 are calculated
canvas.clear_ubo()
p=6
canvas.calculate_ubo_coefficients(p, mixture_parameters, low=2, high=4)
for i in range(1, p+1):
    print(f"Coefficients of order {i}:")
    try:
        print(canvas._ubo["coeffs"][i])
    except KeyError:
        print('missing')
    print()

## Evaluating cost of bitstrings

In [ ]:
# once coefficients are calculated, it is possible to efficiently evaluate the cost of a bitstring

# calculate QUBO coefficients
canvas.clear_ubo()
p=2
canvas.calculate_ubo_coefficients(p, mixture_parameters)

# calcualte cost of random bitstrings
bitstrings = ["101011", "010010", "101100"]
for bs in bitstrings:
    print(f"cost of {bs}:", canvas.calculate_bitstring_cost_from_coefficients(bs))


In [ ]:
# if truncation is not present, the cost can also be calculated directly from a norm
for bs in bitstrings:
    cost = canvas.calculate_bitstring_cost_from_norm(
        bitstring = bs,
        mixture_params = mixture_parameters, 
        norm = Lp_norm, #choice of norm function
        norm_params = [2] #parameter for the norm, in this case it's just the order
    )
    print(f"cost of {bs}:", cost)


# Example of workflow